In [5]:
import os
from telegram.ext  import Updater, CommandHandler, MessageHandler, Filters
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd

In [6]:
import emoji
import inflect
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from num2words import num2words

In [7]:
import mmap
import re

def get_num_lines(file_path):
    fp = open(file_path, "r+")
    buf = mmap.mmap(fp.fileno(), 0)
    lines = 0
    while buf.readline():
        lines += 1
    return lines

In [8]:
assert True

#Small preprocess of the answers

if not os.path.isfile('prepared_answers.txt'):
    question = None
    written = False

    with open("prepared_answers.txt", "w") as fout:
        with open("Otvety.txt", "r") as fin:
            for line in tqdm_notebook(fin):
                if line.startswith("---"):
                    written = False
                    continue
                if not written and question is not None:
                    fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                    written = True
                    question = None
                    continue
                if not written:
                    question = line.strip()
                    continue

In [9]:
import re
import emoji
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from num2words import num2words
import inflect

# Функция для очистки текста
def preprocess_txt(line):
    # HTML-теги: первый шаг - удалить из входного текста все HTML-теги
    clean_text = re.sub('<[^<]+?>', '', line)

    # URL и ссылки: далее - удаляем из текста все URL и ссылки
    clean_text = re.sub(r'http\S+', '', clean_text)

    # Эмоджи и эмотиконы: используем собственную функцию для преобразования эмоджи в текст
    # Важно понимать эмоциональную окраску обрабатываемого текста
    clean_text = emojis_words(clean_text)

    # Приводим все входные данные к нижнему регистру
    clean_text = clean_text.lower()

    # Убираем все пробелы
    # Так как все данные теперь представлены словами - удалим пробелы
    clean_text = re.sub('\s+', ' ', clean_text)

    # Убираем специальные символы: избавляемся от всего, что не является "словами"
    clean_text = re.sub('[^а-яА-ЯёЁ0-9\s]', '', clean_text)

    # Записываем числа прописью: 100 превращается в "сто" (для компьютера)# не работает
    temp = inflect.engine()
    words = []
    for word in clean_text.split():
        if word.isdigit():
            try:
                # Используем num2words только для чисел, которые меньше 100
                num = int(word)
                if num < 100:
                    words.append(num2words(num, lang='ru'))
                else:
                    words.append(word)
            except ValueError:
                words.append(word)
        else:
            words.append(word)

    clean_text = ' '.join(words)

    # Стоп-слова: удаление стоп-слов - это стандартная практика очистки текстов
    stop_words = set(stopwords.words('russian'))
    tokens = word_tokenize(clean_text)
    tokens = [token for token in tokens if token not in stop_words]
    clean_text = ' '.join(tokens)

    # Знаки препинания: далее - удаляем из текста все знаки препинания
    clean_text = re.sub(r'[^\w\s]', '', clean_text)

    # И наконец - возвращаем очищенный текст
    return clean_text

# Функция для преобразования эмоджи в слова
def emojis_words(text):
    # Модуль emoji: преобразование эмоджи в их словесные описания
    clean_text = emoji.demojize(text, delimiters=(" ", " "))

    # Редактирование текста путём замены ":" и "_", а так же - путём добавления пробела между отдельными словами
    clean_text = clean_text.replace(":", "").replace("_", " ")

    return clean_text

def preproc_text(text):
    res = str(text).strip()
    res = re.sub(r"\s+", " ", res)
    return res

def build_data(data_q, data_ans):
    data = []
    for idx, texts in enumerate(data_q):
        question = preproc_text(texts)
        answer = preproc_text(data_ans.iloc[idx])
        res = '\nx:' + question + '\ny:' + answer
        data.append(res)
    return data


In [10]:
assert True

# Preprocess for models fitting

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

In [11]:
file_path_to = "Otvety.txt"
if not os.path.isfile(file_path_to):
    with open("Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            spls = preprocess_txt(line)
            sentences.append(spls)
            c += 1
            if c > 500000:
                break

In [12]:
model_file_path = "ft_model"

if not os.path.isfile(model_file_path):
    sentences = [i for i in sentences if len(i) > 2]
    modelFT = FastText(sentences=sentences, size=100, min_count=1, window=5)
    modelFT.save(model_file_path)
else:
    modelFT = FastText.load(model_file_path)
    ft_index = annoy.AnnoyIndex(100 ,'angular')

In [13]:
index_map = {}
counter = 0

with open("prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT:
                vector_ft += modelFT[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)
            
        counter += 1

ft_index.build(10)
ft_index.save('speaker.ann')

with open("index_speaker.pkl", "wb") as f:
        pickle.dump(index_map, f)

/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_16461/1712945895.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


0it [00:00, ?it/s]

/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_16461/1712945895.py:12: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in modelFT:
/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_16461/1712945895.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_ft += modelFT[word]


In [14]:
ft_index = annoy.AnnoyIndex(100, 'angular')
ft_index.load('speaker.ann') 

True

In [15]:
ft_index.get_nns_by_vector(np.zeros(100), 2)

[12007, 29074]

In [16]:
shop_data = pd.read_csv("ProductsDataset.csv")


shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,юбка детская новая носили разу реале красивей ...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,ботильоны новыепривезены чехии указан размер 4...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,брюки размер 4042 брюки новые знаю мерила поку...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,продам детские шапки продам шапкикажда 200рроз...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,блузка темносиняя сорок размерсостояние отличн...


In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [18]:
idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = ["".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = ["".join(val) for val in shop_data['text'].values]

In [19]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=13)

In [21]:
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression(max_iter=1000).fit(x_train_vec, y_train)


In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.9563213500673615

In [57]:
assert True
import pickle

index_filename = "index_speaker.pkl"

if not os.path.isfile(index_filename):
    with open(index_filename, "wb") as f:
        pickle.dump(index_map, f)
else:
    print(f"Файл '{index_filename}' уже существует. Пропускаем операцию записи.")

Файл 'index_speaker.pkl' уже существует. Пропускаем операцию записи.


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

In [26]:
np.mean(tfidf_vect.idf_)

10.858573842171308

In [27]:
idfs = {v[0]: v[1] for v in zip(tfidf_vect.vocabulary_, tfidf_vect.idf_)}
idfs

{'блузка': 11.247290000252185,
 'одела': 11.247290000252185,
 'разсостояние': 11.247290000252185,
 'новой': 11.247290000252185,
 'вещи': 11.247290000252185,
 'обратитесь': 11.247290000252185,
 'парутройку': 11.247290000252185,
 'банков': 11.247290000252185,
 'правило': 11.247290000252185,
 'налажена': 11.247290000252185,
 'консультация': 11.247290000252185,
 'посетителей': 11.247290000252185,
 'кредитных': 11.247290000252185,
 'инспекторов': 11.247290000252185,
 'программыипотечные': 11.247290000252185,
 'калькуляторы': 11.247290000252185,
 'помощью': 11.247290000252185,
 'рассчитают': 11.247290000252185,
 'размер': 11.247290000252185,
 'сроки': 11.247290000252185,
 'ипотечного': 11.247290000252185,
 'кредита': 11.247290000252185,
 'учётом': 11.247290000252185,
 'ваших': 11.247290000252185,
 'доходов': 11.247290000252185,
 'наличия': 11.247290000252185,
 'первоначального': 10.841824892144022,
 'взноса': 11.247290000252185,
 'самого': 11.247290000252185,
 'начала': 11.247290000252185,
 

In [28]:
ft_index_shop = annoy.AnnoyIndex(100 ,'angular')

midf = np.mean(tfidf_vect.idf_)

index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(100)
    for word in shop_data.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(10)
ft_index_shop.save('shop.ann')

/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_16461/2149195794.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(shop_data))):


  0%|          | 0/35548 [00:00<?, ?it/s]

/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_16461/2149195794.py:13: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in modelFT:
/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_16461/2149195794.py:14: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_ft += modelFT[word] * idfs.get(word, midf)


True

In [29]:
assert True
import pickle

with open("index_shop.pkl", "wb") as f:
    pickle.dump(index_map, f)

In [30]:
def embed_txt(txt, idfs, midf):
    n_ft = 0
    vector_ft = np.zeros(100)
    for word in txt:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    return vector_ft / n_ft

In [31]:
ft_index_shop.get_nns_by_vector(np.zeros(100), 1, include_distances=True)

([289], [1.4142135381698608])

In [32]:
# заменить на свой токен
updater = Updater("6850116261:AAHS8KS3XkquAcFZA4wFZN2SVz0W6RWbQwY", use_context=True) # Токен API к Telegram
dispatcher = updater.dispatcher

def startCommand(update, context):
    context.bot.send_message(chat_id=update.message.chat_id, text='Привет, давай пообщаемся?')

def textMessage(update, context):
    input_txt = preprocess_txt(update.message.text)
    vect = vectorizer.transform([" ".join(input_txt)])
    prediction = lr.predict(vect)
    
    print("Prediction:", prediction)  # Добавим вывод предсказания
    
    # ПРОД
    if prediction[0] == 1:
        print("Entering PROD")  # Добавим вывод, чтобы убедиться, что зашли в этот блок кода
        vect_ft = embed_txt(input_txt, idfs, midf)
        ft_index_shop_val = ft_index_shop.get_nns_by_vector(vect_ft, 5)
        for item in ft_index_shop_val:
            title, image = index_map_shop[item]
            context.bot.send_message(chat_id=update.message.chat_id, text="title: {} image: {}".format(title, image))
        return
    
    # QA
    vect_ft = embed_txt(input_txt, {}, 1)
    ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
    
    # 
    print("Distances:", distances)  # Добавим вывод расстояний
    if distances[0] > 100.5:
        print("Entering Distance Check")  # Добавим вывод, чтобы убедиться, что зашли в этот блок кода
        context.bot.send_message(chat_id=update.message.chat_id, text="Моя твоя не понимать")
        return
    
    # Вопрос-Ответ
    context.bot.send_message(chat_id=update.message.chat_id, text=index_map[ft_index_val[0]])

        
start_command_handler = CommandHandler('start', startCommand)
text_message_handler = MessageHandler(Filters.text, textMessage)
dispatcher.add_handler(start_command_handler)
dispatcher.add_handler(text_message_handler)
updater.start_polling(clean=True)
updater.idle()

/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_16461/1857161328.py:44: TelegramDeprecationWarning: The argument `clean` of `start_polling` is deprecated. Please use `drop_pending_updates` instead.
  updater.start_polling(clean=True)


Prediction: [0.]
Distances: [0.00038705082261003554]


/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_16461/770476893.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in modelFT:
/var/folders/jt/ml43tfs95vn8qnrwr22_cvch0000gp/T/ipykernel_16461/770476893.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  vector_ft += modelFT[word] * idfs.get(word, midf)


Prediction: [0.]
Distances: [0.0]
Prediction: [0.]
Distances: [0.00038586027221754193]
Prediction: [0.]
Distances: [0.0002384667022852227]
Prediction: [0.]
Distances: [0.002932949922978878]
Prediction: [0.]
Distances: [6.667711568297818e-05]
